# Implementation of recursive neural network for electricity usage prediction 1 hour ahead
Applied on case study Oslo Airport Gardermoen for Kjersti Rustad Kvisberg's master thesis autumn 2022.

Requires input with minimum timestamp of electricity measurement and electricity measurement to predict  electricity usage one hour ahead. 
For this master thesis, weather and passenger measurements and calendar information were also used as explanatory variables.

Data is saved in Pandas dataframes, and models are built using Tensorflow and Keras.

## Imports

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from keras.layers import Input, Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.optimizers import RMSprop, SGD, Adam
from keras.models import Sequential
from sklearn.dummy import DummyRegressor

In [ ]:
seed = 221 # customized to use case
np.random.seed(seed)
tf.random.set_seed(seed)

## Loading data

In [ ]:
# Import data set into dataframe using first column as index
df = pd.read_csv('results/data_output/data_cleaned_2017_2021_dummy.csv', 
                 index_col = 0) # customized to use case

In [ ]:
# Convert time stamp index to datetime object for further work
df.index = pd.to_datetime(df.index)

## Inspecting data
Dataframe is checked for missing data (NaN values). Descritive statistics are computed and printed.

In [ ]:
# Print overview of the dataframe to see if there is missing data
df.info()

## Preprocessing data
Dataframe is filtered on relevant time period (April through September 2019).  
Training data are split into target and feature sets, and timesplits with training and validation data for cross validation

In [ ]:
# Select time period
df2 = df.loc[df.index >= '2019-04-01 00:00:00']
df3 = df2.loc[df2.index <= '2019-09-30 23:00:00']

# Specify hourly frequency of measurements
df3 = df3.asfreq('1H')

# Print info about data
df3.info()

In [ ]:
# Print statistics about numerical columns in df
df3.describe()

In [ ]:
# Seperate target column and reshape into correct form 1D array
df_y = df3['Timesverdi'].copy()
y = df_y.values.reshape(-1, 1)

In [ ]:
# Print shape of target array
y.shape

In [ ]:
# Print columns of dataframe
df3.columns

In [ ]:
# Create dataframe with only relevant columns
df_x = df3.drop(['Timesverdi', 'Dato', 'index', 'År' # customized to use case
    ],axis=1)

# Create feature matrix as values from dataframe
df_x = df_x.iloc[:,:]
x = df_x.values

In [ ]:
# Print columns of dataframe with only relevant columns
df_x.columns

In [ ]:
# Print shape of feature matrix
x.shape

In [ ]:
# Save stats
num_features = x.shape[1]
num_targets = y.shape[1]
num_obs = len(x)

In [ ]:
# Specify properties of train, validation and test split and lookback
TRAIN_SIZE = 0.80 # customized to use case
VAL_SIZE = 0.10 # customized to use case
TEST_SIZE = 1 - TRAIN_SIZE - VAL_SIZE
num_train = int(num_obs * TRAIN_SIZE)
num_val = int(num_obs * VAL_SIZE)
num_test = num_obs - num_train - num_val

N_LOOKBACK = 24*7  # length of timestep dimension in Keras for training batches # customized to use case

BATCH_SIZE_TRAIN = 32 # customized to use case
BATCH_SIZE_TEST = 1 # customized to use case

In [ ]:
# Make new feature matrices and target arrays for test, validation and training data

# First num_train objects are training data
x_train = x[0:num_train]
# Taking into account lookback, the next num_val objects are validation data
x_val = x[num_train - N_LOOKBACK:num_train + num_val] 
# Taking into account lookback, the rest are testing data
x_test = x[num_train + num_val - N_LOOKBACK:]

# Similarly for target arrays
y_train_list = y[0:num_train]
y_val_list = y[num_train - N_LOOKBACK:num_train + num_val]
y_test_list = y[num_train + num_val - N_LOOKBACK:]

In [ ]:
# Save time stamps in series for later plotting
train_dt = df3.iloc[0:num_train].index
val_dt = df3.iloc[num_train:num_train + num_val].index
test_dt = df3.iloc[num_train + num_val - N_LOOKBACK:].index

In [ ]:
# Create scaler objects and fit them to training data
x_scaler = MinMaxScaler()
x_scaler = x_scaler.fit(x_train)

y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(y_train_list)

In [ ]:
# Transform feature matrices and target arrays with scalers
x_train_scaled = x_scaler.transform(x_train)
x_val_scaled = x_scaler.transform(x_val)
x_test_scaled = x_scaler.transform(x_test)

y_train_list_scaled = y_scaler.transform(y_train_list)
y_val_list_scaled = y_scaler.transform(y_val_list)
y_test_list_scaled = y_scaler.transform(y_test_list)

In [ ]:
# Check lengths
len(x_train_scaled) + len(x_val_scaled) + len(x_test_scaled) > len(x)

## Compute baseline scores
A "dumb" model is created using Scikit-learns DummyRegressor. The model predicts each measurement to be the mean value of the training data.

In [ ]:
# Create dumb model
dummy_regr = DummyRegressor(strategy='mean')

# Fit to training data
dummy_regr.fit(x_train_scaled, y_train_list_scaled) 

# Predict on test data
dummy_pred = dummy_regr.predict(x_test_scaled) 

# Save measured values in new array for comparison
dummy_true = y_test_list_scaled

In [ ]:
# Transform predictions and measured values back to non-scaled values
dummy_true = y_scaler.inverse_transform(dummy_true.reshape(-1, 1))
dummy_pred = y_scaler.inverse_transform(dummy_pred.reshape(-1, 1))

In [ ]:
# Print scores
print('MSE = ', mean_squared_error(dummy_true, dummy_pred))
print('RMSE = ', np.sqrt(mean_squared_error(dummy_true, dummy_pred)))
print('MAE = ', mean_absolute_error(dummy_true, dummy_pred))
print('MAPE = ', mean_absolute_percentage_error(dummy_true, dummy_pred)*100)

## Build and fit neural network

In [ ]:
# Define model arcitecture
USE_DROPOUTLAYER = True # customized to use case
DROPOUT_SHARE = 0.0 # customized to use case
N_UNITS = 32 # customized to use case
N_EPOCHS = 3*num_features
ACTIVATION = 'ReLU'  # some other options: ['tanh', ' linear ', 'ReLU ']
NUM_LAYERS = 1 # customized to use case
ACTIVATIONS = 'default'

def build_model():
    model = keras.Sequential()

    # LSTM layer with prev. specified number of units and default activations
    model.add(keras.layers.LSTM(units=N_UNITS,
                    input_shape=(N_LOOKBACK, num_features),
                    # return_sequences=True, # uncomment if >1 layer
                    ))
    
    # Adds dropoutlayer with prev. specified percentace of dropout
    if USE_DROPOUTLAYER:
        model.add(keras.layers.Dropout(DROPOUT_SHARE, 
                                       seed=seed))
    
    # Addition of second layer with specified number of units
    if NUM_LAYERS > 1:
        model.add(keras.layers.LSTM(units=int(N_UNITS),
                    # return_sequences=True # uncomment if >2 layers
        ))

        if USE_DROPOUTLAYER:
            model.add(keras.layers.Dropout(DROPOUT_SHARE, seed=seed))
    
    # Addition of third layer with specified number of units
    if NUM_LAYERS > 2:
        model.add(keras.layers.LSTM(units=int(N_UNITS)))
    
        if USE_DROPOUTLAYER:
            model.add(keras.layers.Dropout(DROPOUT_SHARE, seed=seed))
    
    # Add final Dense layer, optionally with not default (linear) activation
    model.add(keras.layers.Dense(1, 
            # activation = ACTIVATION # customized to use case
    ))
    
    return model

In [ ]:
# Print summary of model
build_model().summary()

In [ ]:
# Prepare lists for saving scores and predictions
scores = []
train_predictions = []
val_predictions = []
test_predictions = []

# Save current date and time to be used for model id
date = pd.to_datetime('today').strftime("%d.%m.%Y %H.%M.%S")
MODEL_NAME = date

In [ ]:
# Compilation choices
LOSS = 'mse' # customized to use case
METRICS = ['mse', tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mae', 'mape'] # customized to use case
OPTIMIZER = 'adam' # customized to use case

In [ ]:
# Reshape scaled target arrays
y_train_scaled = np.reshape(y_train_list_scaled[:, 0], newshape=(y_train_list_scaled.shape[0], 1))
y_val_scaled = np.reshape(y_val_list_scaled[:, 0], newshape=(y_val_list_scaled.shape[0], 1))
y_test_scaled = np.reshape(y_test_list_scaled[:, 0], newshape=(y_test_list_scaled.shape[0], 1))

In [ ]:
# Start training and prediction for process for model
model = build_model()
print(model)

# Initialize TimeseriesGenerator from Keras for each part of data
# (PS. deprecated tool, consider switching to generator recommendation in error message)
# For training data
train_data_gen_shuffle = TimeseriesGenerator(x_train_scaled, y_train_scaled, 
            length=N_LOOKBACK, sampling_rate=1, stride=1, 
            batch_size=BATCH_SIZE_TRAIN, 
            shuffle=True)

# For prediction on training data
train_data_gen = TimeseriesGenerator(x_train_scaled, y_train_scaled, 
            length=N_LOOKBACK, sampling_rate=1, stride=1, 
            batch_size=BATCH_SIZE_TRAIN, 
            shuffle=False)

# For validation data during training
val_data_gen = TimeseriesGenerator(x_val_scaled, y_val_scaled, 
            length=N_LOOKBACK, sampling_rate=1, stride=1, 
            batch_size=BATCH_SIZE_TEST, 
            shuffle=True)

# For test data
test_data_gen = TimeseriesGenerator(x_test_scaled, y_test_scaled, 
            length=N_LOOKBACK, sampling_rate=1, stride=1, 
            batch_size=BATCH_SIZE_TEST)

# Define all callbacks for model improvance
USE_EARLYSTOPPING = True
PATIENCE = 20
USE_REDUCELR = True

path_checkpoint = f'results/model_chekpoints_RNN/{date}_checkpoint.keras'  # path to location for saving checkpoint

# Monitor that saves the latest best model regards to validation loss
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                    monitor='val_loss',
                                    verbose=1,
                                    save_weights_only=True,
                                    save_best_only=True)

# Early stopping will end an epoch/training if validation does not improve for PATIENCE amount of steps/epochs .
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=PATIENCE,
                                    verbose=1)

# Reduces learning rate to appropriate number for improved learning
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.1,
                                        min_lr=1e-5,
                                        patience=2,
                                        verbose=1)

# List for collecting all callbacks
callbacks = []  

if USE_EARLYSTOPPING:  # add early stopping if chosen
    callbacks.append(callback_early_stopping)
    callbacks.append(callback_checkpoint)

if USE_REDUCELR:  # add reduceLR if chosen
    callbacks.append(callback_reduce_lr)

# Compile the model using selected settings
model.compile(loss=LOSS,
            metrics=METRICS,
            optimizer=OPTIMIZER)

# Print summary of model
model.summary()

# Train the model with train set generator, while vaildating against validation data
history = model.fit(train_data_gen_shuffle,
                            epochs=N_EPOCHS,
                            steps_per_epoch=10,
                            use_multiprocessing=False,
                            callbacks=callbacks,
                            validation_data=val_data_gen,
                            verbose=1
                            ).history

# Reload the best model from PATIENCE amount of epochs earlier
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

# Create and save loss plots
ax = pd.DataFrame(history)[['loss', 'val_loss']].plot(logy=True, figsize=(10, 5))
fig = ax.get_figure()
fig.savefig(str(MODEL_NAME) + ' _losscurve.png')

ax = pd.DataFrame(history)[['mae', 'val_mae']].plot(figsize=(10, 5))
fig = ax.get_figure()
fig.savefig(str(MODEL_NAME) + '_maecurve.png')

ax = pd.DataFrame(history)[['mape', 'val_mape']].plot(figsize=(10, 5))
fig = ax.get_figure()
fig.savefig(str(MODEL_NAME) + '_mapecurve.png')

# After training, evaluate and do a final forecast
score = model.evaluate(test_data_gen)
scores.append(score)

trainPredict = model.predict(train_data_gen)
valPredict = model.predict(val_data_gen)
testPredict = model.predict(test_data_gen)

train_predictions.append(trainPredict)
val_predictions.append(valPredict)
test_predictions.append(testPredict)

# Checkpoint save of current prediction arrays as .npy files
np.save('results/model_predictions_RNN/'+str(MODEL_NAME)+'_test_predictions', test_predictions)
np.save('results/model_predictions_RNN/'+str(MODEL_NAME)+'_train_predictions', train_predictions)
np.save('results/model_predictions_RNN/'+str(MODEL_NAME)+'_val_predictions', val_predictions)
np.save('results/model_predictions_RNN/'+str(MODEL_NAME)+'_scores', scores)

## Evaluate model

In [ ]:
# Print evaluation of model on training data generator
model.evaluate(train_data_gen)

In [ ]:
# Create arrays of each prediction
test_predictions = np.array(test_predictions)
train_predictions = np.array(train_predictions)
val_predictions = np.array(val_predictions)

In [ ]:
# Print shape of training prediction
train_predictions.shape

In [ ]:
# Print shape of test prediction
test_predictions.shape

In [ ]:
# Reshape prediction arrays
train_predictions = train_predictions.reshape(-1, 1)
test_predictions = test_predictions.reshape(-1, 1)
val_predictions = val_predictions.reshape(-1, 1)

In [ ]:
# Print new shape of training prediction
train_predictions.shape

In [ ]:
# Print new shape of test prediction
test_predictions.shape

In [ ]:
# Inverse transform prediction results to original scale
trainPred = y_scaler.inverse_transform(train_predictions)
testPred = y_scaler.inverse_transform(test_predictions)
valPred = y_scaler.inverse_transform(val_predictions)

In [ ]:
# Print shape of inversed training prediction
trainPred.shape

In [ ]:
# Print shape of inversed test prediction
testPred.shape

In [ ]:
# Print shape of original feature matrix
x.shape

In [ ]:
# Print shape of original target array
y.shape

In [ ]:
# Save true values to arrays
trainTrue = y[0:num_train, 0][-len(trainPred):] # final [] to skip first 168, which are N_LOOKBACK for first prediction in array
valTrue = y[num_train:num_train + num_val, 0]
testTrue = y[num_train+num_val:, 0]

In [ ]:
# Print more shapes
trainTrue.shape

In [ ]:
trainPred.shape

In [ ]:
valTrue.shape

In [ ]:
valPred.shape

In [ ]:
testTrue.shape

In [ ]:
testPred.shape

## Inspect numeric results and parameters

In [ ]:
# Print seed
seed

In [ ]:
# Print model summary again
model.summary()

In [ ]:
# Print info about settings for current model
print(date)
print(TRAIN_SIZE, ':', VAL_SIZE, ':', round(TEST_SIZE,1))
print('do_share:', DROPOUT_SHARE, 'n_units:', N_UNITS, 'n_layers:', NUM_LAYERS)
print('n_epochs:', len(history['val_loss']))
print(', n_lb:', N_LOOKBACK, 'bs:', BATCH_SIZE_TRAIN)
print(', l:', LOSS, ', opt:', OPTIMIZER, ', act: ', ACTIVATIONS)
print('m:', METRICS,)

In [ ]:
# Print training scores
print('MSE = ', round(mean_squared_error(trainTrue,trainPred), 0))
print('RMSE = ', round(np.sqrt(mean_squared_error(trainTrue,trainPred)), 2))
print('MAE = ', round(mean_absolute_error(trainTrue,trainPred), 2))
print('MAPE = ', round(mean_absolute_percentage_error(trainTrue,trainPred)*100, 2))

In [ ]:
# Print validation scores
print('MSE = ', round(mean_squared_error(valTrue,valPred), 0))
print('RMSE = ', round(np.sqrt(mean_squared_error(valTrue,valPred)), 2))
print('MAE = ', round(mean_absolute_error(valTrue,valPred), 2))
print('MAPE = ', round(mean_absolute_percentage_error(valTrue,valPred)*100, 2))

In [ ]:
# Print testing scores
print('MSE = ', round(mean_squared_error(testTrue,testPred), 0))
print('RMSE = ', round(np.sqrt(mean_squared_error(testTrue,testPred)), 2))
print('MAE = ', round(mean_absolute_error(testTrue,testPred), 2))
print('MAPE = ', round(mean_absolute_percentage_error(testTrue,testPred)*100, 2))

## Plot predictions

In [ ]:
# Changeable design parameters
figsize = (10, 3)
dpi = 110

In [ ]:
# List of series of training data time stamps from earlier
trainsett = list(train_dt[N_LOOKBACK:])

In [ ]:
# Check length
len(trainPred)-len(train_dt)

In [ ]:
# Plot results on training data
plt.figure(figsize=figsize, dpi=dpi)
plt.plot(trainsett, trainTrue, color='blue', label='målt', linewidth=1, marker="o", ms=1)
plt.plot(trainsett, trainPred, color='red', label='predikert')
plt.title('Prediksjon av elektrisitetsbruk for hele Oslo lufthavn på treningssett', fontsize=13)
plt.xlabel('Dato')
plt.ylabel('Timeeffekt [kWh/h]')
plt.xticks(trainsett[::450], train_dt[N_LOOKBACK::450].date, rotation=0)
plt.ylim(None, max(trainTrue)+200)
plt.legend(loc=1, ncol=3, fancybox=True)
plt.savefig(f'results/plots_RNN/{date}_train_plott.png', bbox_inches='tight')
plt.show()

In [ ]:
# List of series of test data time stamps from earlier
testsett = list(test_dt[N_LOOKBACK:])

In [ ]:
# Plot results on test data
plt.figure(figsize=figsize, dpi=dpi)
plt.plot(testsett, testTrue,color='blue', label='målt', linewidth=1, marker="o", ms=1)
plt.plot(testsett, testPred, color='red', label='predikert')
plt.title('Prediksjon av elektrisitetsbruk for hele Oslo lufthavn på testsett', fontsize=13)
plt.xlabel('Dato')
plt.ylabel('Timeeffekt [kWh/h]')
plt.xticks(testsett[::60], test_dt[N_LOOKBACK::60].date, rotation=0)
plt.ylim(None, max(testTrue)+800)
plt.legend(loc=1, ncol=3, fancybox=True)
plt.savefig(f'results/plots_RNN/{date}_test_plott.png', bbox_inches='tight')
plt.show ()

In [ ]:
# Playing sound/song when script is finished to remember changing setting/parameters and running again
from playsound import playsound

print("Playing Song using playsound")

playsound("C://Users//Bruker//Music//01 Area Codes.wma") # customized to use case